In [303]:
import matplotlib.pyplot as plt
from matplotlib.patches import ConnectionPatch
import pandas as pd
import numpy as np
from math import ceil

In [304]:
import io
import requests

Content = requests.get("https://raw.githubusercontent.com/flmath-dirty/matrixes_in_erlang/master/results.cvs").content

LoadedTable= pd.read_csv(io.StringIO(Content.decode('utf-8')),skipinitialspace=True,
                  names=['Test type', 'Width', 'Height','Matrix representation','No. of calls', 'Execution time'])

In [305]:
from scipy import stats
from functools import reduce
AggregatedTable = (LoadedTable.groupby(['Test type','Matrix representation'])['Execution time']
                   .apply(list).reset_index(name='Execution Times List'))

In [306]:
AggregatedTable['Harmonic Average'] = AggregatedTable['Execution Times List'].apply(
    lambda x: stats.hmean(x) if (len(x) == 3) else np.inf)


In [307]:
AggregatedTable['Max'] = AggregatedTable['Execution Times List'].apply(
    lambda x: max(x) if (len(x) == 3) else np.inf)

In [308]:
AggregatedTable['Min'] = AggregatedTable['Execution Times List'].apply(min)

In [309]:
AggregatedTable.head()

,Test type,Matrix representation,Execution Times List,Harmonic Average,Max,Min
0,get_value,matrix_as_array,"[59.462, 232.601, 28018.33]",141.828134,2.801833e+04,59.462
1,get_value,matrix_as_big_tuple,"[25.176, 121.824, 12527.693]",62.488602,1.252769e+04,25.176
2,get_value,matrix_as_digraph,"[42.854, 211.419, 19704.339]",106.701803,1.970434e+04,42.854
3,get_value,matrix_as_list_of_lists,"[280.91, 2073.251]",inf,inf,280.910
4,get_value,matrix_as_map,"[36.883, 159.307, 15976.373]",89.679282,1.597637e+04,36.883


In [310]:
GetValue = (
    AggregatedTable
    .loc[AggregatedTable['Test type']=='get_value']
    .loc[:,['Matrix representation','Harmonic Average','Max','Min']]
    )
#GetValue.columns = GetValue.iloc[0]
GetValue = GetValue.set_index('Matrix representation')
#%matplotlib inline

In [17]:
import sawed_bar_plot

ModuleNotFoundError: No module named 'sawed_bar_plot'

In [5]:
sawed_bar_plot(GetValue)

NameError: name 'sawed_bar_plot' is not defined